# Metaclasses and Attributes

- [Item 29: Use Plain Attributes Instead of Get and Set Methods](#Item-29:-Use-Plain-Attributes-Instead-of-Get-and-Set-Methods)
- [Item 30: Consider @property Instead of Refactoring Attributes](#Item-30:-Consider-@property-Instead-of-Refactoring-Attributes)
- [Item 31: Use Descriptors for Reusable @property Methods](#Item-31:-Use-Descriptors-for-Reusable-@property-Methods)
- [Item 32: Use \_\_getattr__, \_\_getattribute__, and \_\_setattr__ for Lazy Attributes](#Item-32:-Use-__getattr__,-__getattribute__,-and-__setattr__-for-Lazy-Attributes)
- [Item 33: Validate Subclasses with Metaclasses](#Item-33:-Validate-Subclasses-with-Metaclasses)
- [Item 34: Register Class Existence with Metaclasses](#Item-34:-Register-Class-Existence-with-Metaclasses)
- [Item 35: Annotate Class Attributes with Metaclasses](#Item-35:-Annotate-Class-Attributes-with-Metaclasses)

## Item 29: Use Plain Attributes Instead of Get and Set Methods

In [ ]:
# Bad practice
# Using setters and getters is not Pythonic
class OldResistor(object):
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms


class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        

r1 = Resistor(50e3)
r1.ohms = 10e2

r1.ohms += 5e2


class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms
        
        
r2 = VoltageResistance(1e3)
print('Before: %5r amps' % r2.current)
r2.voltage = 10
print('After: %5r amps' % r2.current)


class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError('%f ohms must be > 0' % ohms)
        self._ohms = ohms
        

# r3 = BoundedResistance(-5)


class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("can't set attribute")
        self._ohms = ohms
        
r4 = FixedResistance(1e3)
print(f'r4 ohms: {r4.ohms}')
# r4.ohms = 2e3  # Exception


# Don't set other attributes in getter property methods
class MysteriousResistor(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
    
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms
    
r7 = MysteriousResistor(10)
r7.current = 0.01
print('Before: %5r' % r7.voltage)
r7.ohms
print('After: %5r' % r7.voltage)

### Things to Remember

- Define new class interfaces using simple public attributes, and avoid set and get methods.
- Use *@property* to define special behavior when attributes are accessed on your objects, if necessary.
- Follow the rule of least surprise and avoid weird side effects in your *@property* methods.
- Ensure that *@property* methods are fast; do slow or complex work using normal methods.

## Item 30: Consider *@property* Instead of Refactoring Attributes

In [ ]:
from datetime import datetime, timedelta

class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
        
    def __repr__(self):
        return ('Bucket(max=quota=%d, quota_consumed=%d)' %
                (self.max_quota, self.quota_consumed))
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # Quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Quota being filled for the new period
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            # Quota being consumed during the period
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed = delta
            
    
def fill(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period.delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount
    

def deduct(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        return False
    if bucket.quota - amount < 0:
        return False
    bucket.quota -= amount
    return True


bucket = Bucket(100)
print('Initial', bucket)
bucket.quota = 100
print('Fill', bucket)
bucket.quota -= 30
print('deduct', bucket)
bucket.quota -= 20
print('deduct', bucket)
bucket.quota -= 50
print('deduct', bucket)
bucket.quota += 500
print('deduct', bucket)

## Item 31: Use Descriptors for Reusable *@property* Methods


In [ ]:
# Wrong implementation
class Grade(object):
    def __init__(self):
        self._value = 0
        
    def __get__(self, instance, instance_type):
        return self._value
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value = value
        
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()


first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

second_exam = Exam()
second_exam.writing_grade = 75
print('Second', second_exam.writing_grade, 'is right')
print('First', first_exam.writing_grade, 'is wrong')

In [ ]:
from weakref import WeakKeyDictionary

class Grade(object):
    def __init__(self):
        # Problem: Memory leaks
        # self._value = {}
        
        # Improvement: Use WeakKeyDictionary
        self._value = WeakKeyDictionary()
        
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return self._value.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value[instance] = value
        
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()


first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

second_exam = Exam()
second_exam.writing_grade = 75
print('Second', second_exam.writing_grade, 'is right')
print('First', first_exam.writing_grade, 'is right')

### Things to Remember

- Reuse the behavior and validation of @property methods by defining your own descriptor classes.
- Use **WeakKeyDictionary** to ensure that your descriptor classes don't cause memory leaks.
- Don't get bogged down trying to understand exactly how \_\_getattribute__ uses the descriptor protocol for getting and setting attributes.

## Item 32: Use *\_\_getattr__*, *\_\_getattribute__*, and *\_\_setattr__* for Lazy Attributes

In [ ]:
class LazyDB(object):
    def __init__(self):
        self.exists = 5
        
    def __getattr__(self, name):
        value = 'Value for %s' % name
        setattr(self, name, value)
        return value
    

data = LazyDB()
print('Before:', data.__dict__)
print('foo:   ', data.foo)
print('After: ', data.__dict__)


class LoggingLazyDB(LazyDB):
    def __getattr__(self, name):
        print('Called __getattr__(%s)' % name)
        return super().__getattr__(name)
    
    
data = LoggingLazyDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)

In [ ]:
class ValidatingDB(object):
    def __init__(self):
        self.exists = 5
        
    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        try:
            return super().__getattribute__(name)
        except AttributeError:
            value = 'Value for %s' % name
            setattr(self, name, value)
            return value
        
    def __getattr__(self, name):
        print('Called __getattr__(%s)' % name)
        return super().__getattr__(name)


data = ValidatingDB()
print('exists:', data.exists)
print('foo:   ', hasattr(data, 'foo'))
# print('foo:   ', data.foo)
# print('foo:   ', data.foo)

In [ ]:
class BrokenDictionaryDB(object):
    def __init__(self, data):
        self._data = {}
        
    def __getattribute__(self, name):
        print(f'Called __getattribute__({name})')
        # Stack overflow
        return self._data[name]
    

class DictionaryDB(object):
    def __init__(self, data):
        self._data = data
        
    def __getattribute__(self, name):
        data_dict = super().__getattribute__('_data')
        return data_dict[name]

### Things to Remember

- Use *\_\_getattr__* and *\_\_setattr__* to lazily load and save attributes for an object.
- Understand that *\_\_getattr__* only gets called once when accessing a missing attribute, whereas *\_\_getattribute__* gets called every time an attribute is accessed.
- Avoid infinite recursion in *\_\_getattribute__* and *\_\_setattr__* by using methods for *super()* (i.e., the *object* class) to access instance attributes directly.

## Item 33: Validate Subclasses with Metaclasses

One of the simplest applications of metaclasses is verifying that a class was defined correctly.

In [ ]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print((meta, name, bases, class_dict))
        return type.__new__(meta, name, bases, class_dict)
    
class MyClass(object, metaclass=Meta):
    stuff = 123
    
    def __init__(self, x):
        self.x = x
    
    def foo(self):
        pass

In [ ]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        if bases != (object,):
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)
    

class Polygon(object, metaclass=ValidatePolygon):
    sides = None  # Specified by subclasses
    
    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180
    

class Triangle(Polygon):
    sides = 3
    
class Line(Polygon):
    sides = 1

### Things to Remember

- Use metaclasses to ensure that subclasses are well formed at the time they are defined, before objects of their type are constructed.
- Metaclasses have slightly different syntax in Python 2 vs. Python 3
- The *\_\_new__* method of metaclasses is run after the class statement's entire body has been processed.

## Item 34: Register Class Existence with Metaclasses

In [ ]:
import json

registry = {}

def register_class(target_class):
    registry[target_class.__name__] = target_class


def deserialize(data):
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args'])


class Meta(type):
    def __new__(meta, name, bases, class_dict):
        cls = type.__new__(meta, name, bases, class_dict)
        #  Register automatically
        register_class(cls)
        return cls
    

class BetterSerializable(object):
    def __init__(self, *args):
        self.args = args
        
    def serialize(self):
        return json.dumps({
            'class': self.__class__.__name__,
            'args' : self.args,
        })
    
    def __repr__(self):
        return f'{self.__class__.__name__}{self.args}'


class RegisteredSerializable(BetterSerializable,
                             metaclass=Meta):
    pass


class Vector3D(RegisteredSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x, self.y, self.z = x, y, z

        
v3 = Vector3D(10, -7, 3)
print('Before: ', v3)
data = v3.serialize()
print('Serialized:', data)
print('After: ', deserialize(data))

### Things to Remember

- Class registration is a helpful pattern for building modular Python programs.
- Metaclasses let you run registration code automatically each time your base class is subclassed in a program.
- Using metaclasses for class registration avoids errors by ensuring that you never miss a registration calss.

## Item 35: Annotate Class Attributes with Metaclasses

In [ ]:
class Field(object):
    def __init__(self):
        # These will be assigned by the metaclass
        self.name = None
        self.internal_name = None
        
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return getattr(instance, self.internal_name, '')
    
    def __set__(self, instance, value):
        setattr(instance, self.internal_name, value)

class Meta(type):
    def __new__(meta, name, bases, class_dict):
        for key, value in class_dict.items():
            if isinstance(value, Field):
                value.name = key
                value.internal_name = '_' + key
        cls = type.__new__(meta, name, bases, class_dict)
        return cls
    
class DatabaseRow(object, metaclass=Meta):
    pass

class BetterCustomer(DatabaseRow):
    first_name = Field()
    last_name = Field()
    
foo = BetterCustomer()
print('Before:', repr(foo.first_name), foo.__dict__)
foo.first_name = 'Euler'
print('After: ', repr(foo.first_name), foo.__dict__)

### Things to Remember

- Metaclasses enable you to modify a class's attributes before the class is fully defined.
- Dscriptors and metaclasses make a powerful combination for declarative behavior and runtime introspection.
- You an avoid both memory leaks and the *weakref* module by using metaclasses along with descriptors.